In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import json
import time

colors = {
    "GB": px.colors.qualitative.T10[0],
    "GB_translucent": "rgba(76,120,168,0.5)",
    "Nordic": px.colors.qualitative.T10[1],
    "Nordic_translucent": "rgba(245,133,24,0.5)",
    "CE": px.colors.qualitative.T10[2],
    "CE_translucent": "rgba(228,87,85,0.5)",
    "Baltic": px.colors.qualitative.T10[3],
    "Baltic_translucent": "rgba(114,183,178,0.5)",
    "IE": px.colors.qualitative.T10[4],
    "IE_translucent": "rgba(84,162,75,0.5)",
}


# Figure 1a

In [27]:
ind = pd.to_datetime("2016-04-04 22:00:00+02:00")

freq = (
    -50
    + pd.read_hdf(
        "../../Frequency_data_preparation/TransnetBW/cleansed_2015-01-01_to_2019-12-31.h5"
    ).loc["2015-01-01":"2018-12-31"]
)
smoothed_freq_dt = (
    freq.loc[ind - pd.DateOffset(minutes=60) : ind + pd.DateOffset(minutes=60)]
    .diff()
    .rolling(60, center=True)
    .mean()
)
smoothed_freq_dt = smoothed_freq_dt.loc[
    ind - pd.DateOffset(minutes=1) : ind + pd.DateOffset(minutes=1)
]
freq = freq.loc[ind - pd.DateOffset(minutes=5) : ind + pd.DateOffset(minutes=65)]
arg_ext = np.argmax(smoothed_freq_dt.abs().values)
rocof = smoothed_freq_dt.iloc[arg_ext]

f = lambda x, a, b: a * x + b

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=freq.index, y=freq, line_color="black", line_width=1, showlegend=False
    ),
)
fig.add_trace(
    go.Scatter(
        x=freq.index[300:-300],
        y=freq[300:-300],
        line_color="black",
        line_width=1,
        fill="tozeroy",
        fillcolor=px.colors.qualitative.T10[3],
        showlegend=False,
        name="Integral",
    )
)
fig.add_trace(
    go.Scatter(
        x=freq.index[:600],
        y=f(np.arange(-5 * 60, 5 * 60), rocof, 0),
        line_width=3,
        line_color=px.colors.qualitative.T10[0],
        name="RoCoF  ",
        legendgroup=1,
        legendgrouptitle_text=" ",
    ),
)
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="lines",
        line_color=px.colors.qualitative.T10[1],
        line_width=3,
        name="Nadir   ",
        legendgroup=2,
        legendgrouptitle_text=" ",
    )
)
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="lines",
        line_color=px.colors.qualitative.T10[2],
        line_width=3,
        fill="tozeroy",
        fillcolor="rgba(228,87,85,0.1)",
        name="MSD    ",
        legendgroup=3,
        legendgrouptitle_text=" ",
    )
)
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="lines",
        line_color="black",
        line_width=1,
        fill="tozeroy",
        fillcolor=px.colors.qualitative.T10[3],
        name="Integral    ",
        legendgroup=4,
        legendgrouptitle_text=" ",
    )
)

# Nadir
nadir = freq.min()
fig.add_hline(y=nadir, line_color=px.colors.qualitative.T10[1], line_width=3)

msd = (freq[300:-300] ** 2).mean()
# fig.add_hline(y=np.sqrt(msd))
# fig.add_hline(y=-np.sqrt(msd))
fig.add_shape(
    type="rect",
    x0=freq.index[0] - pd.DateOffset(minutes=60),
    y0=np.sqrt(msd),
    x1=freq.index[-1] + pd.DateOffset(minutes=60),
    y1=-np.sqrt(msd),
    line=dict(
        color=px.colors.qualitative.T10[2],
        width=3,
    ),
    layer="below",
    fillcolor="rgba(228,87,85,0.1)",
)


fig.update_layout(
    {
        "yaxis": {
            "title": "Frequency deviation [Hz]",
            "title_standoff": 25,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
            "range": [-0.1, 0.099],
            "dtick": 0.05,
        },
        "xaxis": {
            "title_standoff": 25,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": True,
            "ticks": "outside",
            "showline": True,
            "range": [freq.index[0], freq.index[-1]],
        },
    }
)

fig.update_layout(
    template="plotly_white",
    width=620,
    height=760,
    legend_orientation="h",
    legend_y=-0.24,
)

fig.update_layout(font=dict(size=20, color="black"))
fig.update_annotations(font_size=20)
fig.update_layout(legend_font_size=20)
fig.update_layout(legend_y=-0.3)
fig.update_layout(margin_t=1)
fig.update_layout(margin_b=378)
fig.update_layout(margin_r=20)

fig.add_annotation(
    x=-0.2,
    y=1.05,
    text="<b>a</b>",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    xref="paper",
    yref="y domain",
    font_size=50,
)

print("creating plot...")
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(fig, "fig1a.pdf", format="pdf", validate=False, engine="kaleido")
time.sleep(1)
pio.write_image(fig, "fig1a.pdf", format="pdf", validate=False, engine="kaleido")
scope._shutdown_kaleido()


creating plot...


# Figure 1b

In [23]:
fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        mode="markers",
        lon=[None],
        lat=[None],
        marker={"size": 100},
        marker_symbol="square",
        marker_color=colors["GB_translucent"],
        name="GB",
        legendgrouptitle_text="<b>Sync Area</b>",
        legendgroup=9,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="markers",
        lon=[None],
        lat=[None],
        marker={"size": 100},
        marker_symbol="square",
        marker_color=colors["Nordic_translucent"],
        name="Nordic",
        legendgroup=9,
    )
)
fig.add_trace(
    go.Scattergeo(
        lon=[None], lat=[None], marker_color="rgba(0,0,0,0)", name=" ", legendgroup=9
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="markers",
        lon=[None],
        lat=[None],
        marker={"size": 100},
        marker_symbol="square",
        marker_color=colors["CE_translucent"],
        name="CE",
        legendgroup=10,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="markers",
        lon=[None],
        lat=[None],
        marker={"size": 100},
        marker_symbol="square",
        marker_color=colors["Baltic_translucent"],
        name="Baltic",
        legendgrouptitle_text=" ",
        legendgroup=10,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="markers",
        lon=[None],
        lat=[None],
        marker={"size": 100},
        marker_symbol="square",
        marker_color=colors["IE_translucent"],
        name="Ireland",
        legendgrouptitle_text="                                        ",
        legendgroup=11,
    )
)

fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[0.717, 4.021],
        lat=[51.44, 51.9575],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Set2[4],
        marker_color=px.colors.qualitative.Set2[4],
        text="Britned",
        name="Britned",
        legendgrouptitle_text="<b>GB/CE                     </b>",
        legendgroup=1,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[1.784722, 0.975556],
        lat=[50.903056, 51.105833],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Set2[0],
        marker_color=px.colors.qualitative.Set2[0],
        name="IFA",
        legendgroup=1,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[-3.072778, -6.5675],
        lat=[53.227222, 53.471111],
        marker={"size": 5},
        line_width=2,
        line_color=px.colors.qualitative.Set3[5],
        marker_color=px.colors.qualitative.Set3[5],
        name="EWIC",
        showlegend=False,
        legendgrouptitle_text="<b>GB/IE                     </b>",
        legendgroup=2,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[-4.980556, -5.769722],
        lat=[
            55.069444,
            54.842778,
        ],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Set2[1],
        marker_color=px.colors.qualitative.Set2[1],
        name="Moyle",
        legendgroup=2,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[24.551667, 24.560278],
        lat=[60.203889, 59.384722],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Safe[8],
        marker_color=px.colors.qualitative.Safe[8],
        name="Estlink",
        legendgroup=3,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lon=[21.256667, 15.854167],
        lat=[55.681667, 56.767778],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.G10[8],
        marker_color=px.colors.qualitative.G10[8],
        name="Nordbalt",
        legendgrouptitle_text="<b>Nordic/Baltic</b>",
        legendgroup=3,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[57.062778, 57.606667],
        lon=[10.09, 12.111111],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Light24[5],
        marker_color=px.colors.qualitative.Light24[5],
        name="Kontiskan",
        legendgrouptitle_text="<b>Nordic/CE</b>",
        legendgroup=5,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[55.42857, 55.66104],
        lon=[10.40720, 11.08446],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[22],
        marker_color=px.colors.qualitative.Dark24[22],
        name="Storebaelt",
        legendgroup=5,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[58.26, 56.478889],
        lon=[7.898611, 9.566944],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[10],
        marker_color=px.colors.qualitative.Dark24[10],
        legendgrouptitle_text=" ",
        name="Skagerrak",
        legendgroup=6,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[53.434444, 58.282778],
        lon=[6.865833, 6.865278],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[9],
        marker_color=px.colors.qualitative.Dark24[9],
        name="Norned",
        legendgroup=6,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[54.502111, 56.152972],
        lon=[16.891222, 14.8415],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[0],
        marker_color=px.colors.qualitative.Dark24[0],
        name="Swepol",
        legendgrouptitle_text=" ",
        legendgroup=7,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[53.896056, 55.500306],
        lon=[10.802417, 13.145833],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[15],
        marker_color=px.colors.qualitative.Dark24[15],
        name="Baltic Cable",
        legendgroup=7,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines",
        lat=[55.45025, 54.100889],
        lon=[12.007417, 12.217056],
        marker={"size": 5},
        line_width=2,
        showlegend=False,
        line_color=px.colors.qualitative.Dark24[19],
        marker_color=px.colors.qualitative.Dark24[19],
        name="Kontek",
        legendgrouptitle_text=" ",
        legendgroup=8,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lon=[0.717, 4.021],
        lat=[51.44, 51.9575],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Set2[4],
        marker_color=px.colors.qualitative.Set2[4],
        text="Britned",
        name="Britned",
        legendgrouptitle_text="<b>GB/CE                     </b>",
        legendgroup=1,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        lon=[1.784722, 0.975556],
        lat=[50.903056, 51.105833],
        opacity=0.6,
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Set2[0],
        marker_color=px.colors.qualitative.Set2[0],
        name="IFA",
        legendgroup=1,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        lon=[-3.072778, -6.5675],
        lat=[53.227222, 53.471111],
        opacity=0.6,
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Set3[5],
        marker_color=px.colors.qualitative.Set3[5],
        name="EWIC",
        legendgrouptitle_text="<b>GB/IE                     </b>",
        legendgroup=2,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lon=[-4.980556, -5.769722],
        lat=[
            55.069444,
            54.842778,
        ],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Set2[1],
        marker_color=px.colors.qualitative.Set2[1],
        name="Moyle",
        legendgroup=2,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lon=[24.551667, 24.560278],
        lat=[60.203889, 59.384722],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Safe[8],
        marker_color=px.colors.qualitative.Safe[8],
        name="Estlink",
        legendgroup=3,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lon=[21.256667, 15.854167],
        lat=[55.681667, 56.767778],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.G10[8],
        marker_color=px.colors.qualitative.G10[8],
        name="Nordbalt",
        legendgrouptitle_text="<b>Nordic/Baltic</b>",
        legendgroup=3,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[57.062778, 57.606667],
        lon=[10.09, 12.111111],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Light24[5],
        marker_color=px.colors.qualitative.Light24[5],
        name="Kontiskan",
        legendgrouptitle_text="<b>Nordic/CE</b>",
        legendgroup=5,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[55.42857, 55.66104],
        lon=[10.40720, 11.08446],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[22],
        marker_color=px.colors.qualitative.Dark24[22],
        name="Storebaelt",
        legendgroup=5,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[58.26, 56.478889],
        lon=[7.898611, 9.566944],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[10],
        marker_color=px.colors.qualitative.Dark24[10],
        legendgrouptitle_text=" ",
        name="Skagerrak",
        legendgroup=6,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[53.434444, 58.282778],
        lon=[6.865833, 6.865278],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[9],
        marker_color=px.colors.qualitative.Dark24[9],
        name="Norned",
        legendgroup=6,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[54.502111, 56.152972],
        lon=[16.891222, 14.8415],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[0],
        marker_color=px.colors.qualitative.Dark24[0],
        name="Swepol",
        legendgrouptitle_text=" ",
        legendgroup=7,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[53.896056, 55.500306],
        lon=[10.802417, 13.145833],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[15],
        marker_color=px.colors.qualitative.Dark24[15],
        name="Baltic Cable",
        legendgroup=7,
    )
)
fig.add_trace(
    go.Scattergeo(
        mode="lines+markers",
        opacity=0.6,
        lat=[55.45025, 54.100889],
        lon=[12.007417, 12.217056],
        marker={"size": 8},
        line_width=8,
        line_color=px.colors.qualitative.Dark24[19],
        marker_color=px.colors.qualitative.Dark24[19],
        name="Kontek",
        legendgrouptitle_text=" ",
        legendgroup=8,
    )
)


fig.update_geos(
    resolution=50,
    scope="europe",
    showcoastlines=False,
    coastlinecolor="black",
    showland=True,
    landcolor="white",
    showocean=False,
    oceancolor="white",
    showlakes=False,
    lakecolor="white",
    showcountries=False,
    countrycolor="white",
    lataxis_range=[50, 62],
    lonaxis_range=[-10.8, 25.4],
    framecolor="black",
    framewidth=5,
    showframe=True,
)

fig.update_layout(
    margin={"l": 1, "t": 1, "b": 378, "r": 1},
    height=760,
    width=620,
)


fig.update_layout(font=dict(size=20, color="black"))
fig.update_annotations(font_size=20)
fig.update_layout(legend_font_size=20)
fig.update_layout(legend_orientation="h")

fig.add_shape(
    type="rect",
    xref="paper",
    yref="paper",
    x0=0,
    y0=0,
    x1=1,
    y1=1,
    line=dict(
        color="black",
        width=1,
    ),
)

fig.add_annotation(
    x=0.01,
    y=1.02,
    text="<b>b</b>",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    xref="x domain",
    yref="paper",
    font_size=50,
)

with open("scalerank.geojson") as f:
    regions = json.load(f)


CE = [
    "Luxembourg",
    "Italy",
    "Spain",
    "Portugal",
    "Slovakia",
    "Austria",
    "Hungary",
    "Switzerland",
    "Poland",
    "Belgium",
    "Czech Republic",
    "Luxemburg",
    "France",
    "Germany",
    "Netherlands",
    "Denmark1-2",
    "Denmark3-1",
    "Liechtenstein",
    "Andorra",
]
IE = ["Ireland", "Northern Ireland"]
GB = ["England", "Wales", "Scotland"]
Baltic = [
    "Estonia",
    "Lithuania",
    "Latvia",
    "Russia",
    "Belarus",
    "Ukraine",
    "Kaliningrad",
]
Nordic = ["Denmark1-1", "Denmark5-2", "Sweden", "Finland", "Norway"]
fig.add_trace(
    go.Choropleth(
        locationmode="geojson-id",
        geojson=regions,
        z=[1] * len(CE),
        locations=CE,
        featureidkey="properties.sr_subunit",
        colorscale=[(0, colors["CE_translucent"]), (1, colors["CE_translucent"])],
        showscale=False,
        marker_line_width=1,
        marker_line_color="white",
    )
)
fig.add_trace(
    go.Choropleth(
        locationmode="geojson-id",
        geojson=regions,
        z=[1] * len(IE),
        locations=IE,
        featureidkey="properties.sr_subunit",
        colorscale=[(0, colors["IE_translucent"]), (1, colors["IE_translucent"])],
        showscale=False,
        marker_line_width=1,
        marker_line_color="white",
    )
)
fig.add_trace(
    go.Choropleth(
        locationmode="geojson-id",
        geojson=regions,
        z=[1] * len(GB),
        locations=GB,
        featureidkey="properties.sr_subunit",
        colorscale=[(0, colors["GB_translucent"]), (1, colors["GB_translucent"])],
        showscale=False,
        marker_line_width=0,
        marker_line_color=colors["GB_translucent"],
    )
)
fig.add_trace(
    go.Choropleth(
        locationmode="geojson-id",
        geojson=regions,
        z=[1] * len(Baltic),
        locations=Baltic,
        featureidkey="properties.sr_subunit",
        colorscale=[
            (0, colors["Baltic_translucent"]),
            (1, colors["Baltic_translucent"]),
        ],
        showscale=False,
        marker_line_width=1,
        marker_line_color="white",
    )
)
fig.add_trace(
    go.Choropleth(
        locationmode="geojson-id",
        geojson=regions,
        z=[1] * len(Nordic),
        locations=Nordic,
        featureidkey="properties.sr_subunit",
        colorscale=[
            (0, colors["Nordic_translucent"]),
            (1, colors["Nordic_translucent"]),
        ],
        showscale=False,
        marker_line_width=1,
        marker_line_color="white",
    )
)

print("creating plot...")
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(fig, "fig1b.pdf", format="pdf", validate=False, engine="kaleido")
time.sleep(1)
pio.write_image(fig, "fig1b.pdf", format="pdf", validate=False, engine="kaleido")
scope._shutdown_kaleido()


creating plot...


In [26]:
px.line(pd.read_hdf('../data/flow_input/Nordic-CE/Kontiskan/version_2022-06-24/X_test_full_links.h5').sort_index().f_integral_Nordic)